## Importing Dependencies

In [1]:
import requests
from htmldate import find_date
from datetime import datetime
import requests
import re
from bs4 import BeautifulSoup
import ssl
from tqdm import tqdm
from textblob import TextBlob
import openai
import multiprocessing
from time import time

from html_extractor import *
from get_suburls import *
from openai_func import *
from get_date import *
from parallel import *

from time import sleep

from keyword_extraction import keyword_extractor_paragraph as kep

## Extracting sub urls

In [2]:
urls_list = ["https://www.khaleejtimes.com" , "https://www.indiatoday.in"]
urls_list_str = ",".join(urls_list)

keywords = "gaza,israel,hamas,idf"

scraper = WebScraper2(sub_url_size = 3 , keywords = keywords)
                        # Integration with DB will make it faster in future, as fetching is much faster than scrapping.
inside_urls, failed_fetch, sub_url_size, total_size = scraper.get_suburls2(urls_list_str)

# print("Inside URLs:", inside_urls)
print("Failed Fetch:", failed_fetch)
print("Splits:", len(inside_urls))
print("Tree size:", total_size)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 66/66 [00:23<00:00,  2.79it/s]


Failed Fetch: 0
Splits: 4
Tree size: 181


## Joining sub urls into one single list

In [3]:
website_urls = [item for sublist in list(inside_urls.values()) for item in sublist]
print(len(website_urls))


181


## Testing Date Fetching

Need to integrate Mongo 

In [4]:
fetch_date_from_url(website_urls[3])

('https://www.indiatoday.in/topic/israel', '12-09-2019')

## Creating {url : html content} dictionary

In [4]:
url_html_extracted = get_html(website_urls)
# url_html_extracted

100%|██████████| 181/181 [00:59<00:00,  3.03it/s]


## Keyword extraction performed on above dictionary

In [5]:
url_extracted_html = kep(website_content = url_html_extracted[0], keywords = keywords, filter_by_amount = 60)

# url_extracted_html

100%|██████████| 118/118 [00:00<00:00, 8663.04it/s]


## Converting dictionary to list of tuple pairs, for implementation of batches

In [6]:
content_list = [(key,value[:2000]) for key, value in url_extracted_html.items()] # 1000 is temporary until tokenier function is not set up
# content_list

## Cutting the above list fo batches of batch size MAX_CONTENT

In [7]:
MAX_CONTENT = 5

content_list_complete = []

iterations = len(content_list) // MAX_CONTENT


for i in range(iterations):
    sub_content_list = content_list[MAX_CONTENT * i: MAX_CONTENT * (i + 1)]
    content_list_complete.append(sub_content_list)

# Handle remaining elements after the loop
remaining_elements = content_list[MAX_CONTENT * iterations:]
if remaining_elements:
    iterations += 1
    content_list_complete.append(remaining_elements)

len(content_list_complete)

24

## Openai's api

### Non parallel execution of 1 api

In [21]:
from time import sleep
start = time()

question = "Summary of situation in gaza"

response_complete = ''
for data_idx in range(10):

    prompt = f""" 
        Data is in the form of tuples inside list: {content_list_complete[data_idx]} \n\n\n 
        Question: {question} \n\n\n
        Method of reply: 100 - 200 word sentences, clear reply,
        provide url if neccessary.
        """
    
    if data_idx % 6 == 0:
        sleep(20)

    # response = get_completion(prompt)
    # response_complete += response + "\n\n"
    print(f"Batch {data_idx + 1} out of {iterations} completed ")

end = time()

print(f"Executed in {end-start:.2f}s")
    

Batch 1 out of 24 completed 
Batch 2 out of 24 completed 
Batch 3 out of 24 completed 
Batch 4 out of 24 completed 
Batch 5 out of 24 completed 
Batch 6 out of 24 completed 
Batch 7 out of 24 completed 
Batch 8 out of 24 completed 
Batch 9 out of 24 completed 
Batch 10 out of 24 completed 
Executed in 216.86s


### Parallel Execution for 2 api

In [18]:
start = time()

complete_result_of_openai = ""
question_to_pass = "status of war in gaza"

count = 0

NUM_OF_API = 2

for i in range(0, 10, NUM_OF_API):

    if count % 6 == 0:
        sleep(20)
    

    result = ''

    result_queue1 = multiprocessing.Queue()
    result_queue2 = multiprocessing.Queue()


    process1 = multiprocessing.Process(target=gpt1, args=(question_to_pass, content_list_complete, i, result_queue1))
    process2 = multiprocessing.Process(target=gpt2, args=(question_to_pass, content_list_complete, i+1, result_queue2))


    # Start processes
    process1.start()
    process2.start()

    # Wait for processes to finish
    process1.join()
    process2.join()


    result1 = result_queue1.get()
    result2 = result_queue2.get()

    # Rest of your code remains unchanged
    result = result1 + "\n\n" + result2 + "\n\n"
    complete_result_of_openai += result

    print(f"Batch {i+1} - {i+NUM_OF_API} executed out of {len(content_list_complete)}")
    count += 1
complete_result_of_openai

end = time()

print(f"Executed in {end-start:.2f}s")


Batch 1 - 2 executed out of 24
Batch 3 - 4 executed out of 24
Batch 5 - 6 executed out of 24
Batch 7 - 8 executed out of 24
Batch 9 - 10 executed out of 24
Executed in 98.97s


In [22]:
216/98

2.204081632653061

In [23]:
with open("Output_gaza_parallel.txt" , "w") as f:
    f.write(complete_result_of_openai)

In [18]:
with open("Output_gaza_parallel_summary.txt" , "w") as f:
    f.write(complete_result_of_openai)

In [63]:
with open("Output_gaza.txt" , "w") as f:
    f.write(response_complete)

{'search_metadata': {'id': '65e3f179ca968fce70b728f5',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/a1f806d995d2e248/65e3f179ca968fce70b728f5.json',
  'created_at': '2024-03-03 03:41:45 UTC',
  'processed_at': '2024-03-03 03:41:45 UTC',
  'google_url': 'https://www.google.com/search?q=cat&oq=cat&num=1&sourceid=chrome&ie=UTF-8',
  'raw_html_file': 'https://serpapi.com/searches/a1f806d995d2e248/65e3f179ca968fce70b728f5.html',
  'total_time_taken': 1.17},
 'search_parameters': {'engine': 'google',
  'q': 'cat',
  'google_domain': 'google.com',
  'num': '1',
  'device': 'desktop'},
 'search_information': {'query_displayed': 'cat',
  'organic_results_state': 'Results for exact spelling'},
 'knowledge_graph': {'title': 'About Cat',
  'type': 'Animal',
  'kgmid': '/m/01yrx',
  'knowledge_graph_search_link': 'https://www.google.com/search?kgmid=/m/01yrx&hl=en-US&hl=Cat',
  'serpapi_knowledge_graph_search_link': 'https://serpapi.com/search.json?device=desktop&engine=g

In [2]:
from serpapi import GoogleSearch

from_date = '2019-02-22'
to_date = '2024-03-03'

params = {
  "q": 'Ambani Anant Ambani marriage site:http://indiatoday.in OR site:http://timesofindia.indiatimes.com',
  "tbm": "nws",
  "api_key": "d7555dcb6698cf4004b757eeade1ac86a425b101cb972691dc757282f353d9f6",
  "tbs": f"cdr:1,cd_min:{from_date.replace('-', '/')},cd_max:{to_date.replace('-', '/')}",
  'num' : 100
}

search = GoogleSearch(params)
results = search.get_dict()
news_results = results["news_results"]
news_results

KeyError: 'news_results'

In [3]:
results

{'search_metadata': {'id': '65e3fcbf477c0e45c476fabf',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/1aaeaac6d67b6acc/65e3fcbf477c0e45c476fabf.json',
  'created_at': '2024-03-03 04:29:51 UTC',
  'processed_at': '2024-03-03 04:29:51 UTC',
  'google_url': 'https://www.google.com/search?q=Ambani+Anant+Ambani+marriage+site%3Ahttp%3A%2F%2Findiatoday.in+OR+site%3Ahttp%3A%2F%2Ftimesofindia.indiatimes.com&oq=Ambani+Anant+Ambani+marriage+site%3Ahttp%3A%2F%2Findiatoday.in+OR+site%3Ahttp%3A%2F%2Ftimesofindia.indiatimes.com&num=100&tbm=nws&tbs=cdr:1,cd_min:2019/02/22,cd_max:2024/03/03&sourceid=chrome&ie=UTF-8',
  'raw_html_file': 'https://serpapi.com/searches/1aaeaac6d67b6acc/65e3fcbf477c0e45c476fabf.html',
  'total_time_taken': 0.83},
 'search_parameters': {'engine': 'google',
  'q': 'Ambani Anant Ambani marriage site:http://indiatoday.in OR site:http://timesofindia.indiatimes.com',
  'google_domain': 'google.com',
  'num': '100',
  'device': 'desktop',
  'tbm': 'nws',
 

In [1]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email(sender_email, sender_password, receiver_email, subject, message):
    # Set up the MIME
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Attach the message to the email
    msg.attach(MIMEText(message, 'plain'))

    # Create SMTP session
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()

    # Login to the SMTP server
    server.login(sender_email, sender_password)

    # Send email
    server.sendmail(sender_email, receiver_email, msg.as_string())

    # Terminate the SMTP session
    server.quit()

In [ ]:
sender_email = "trolldude326@gmail.com"
sender_password = "mafazu03"

receiver_email = "petraoil.prod@gmail.com"
subject = "test"
message = "Hello world!"

send_email(sender_email, sender_password, receiver_email, subject, message)

In [26]:
import smtplib
import ssl
import certifi
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage

port = 587  # For starttls
smtp_server = "smtp.gmail.com"
sender_email = "bat463660@gmail.com"
receiver_email = "petraoil.prod@gmail.com"
password = "bygc aape tnem adev"

# Create a multipart message
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = "Subject: Hello"

# Add body to email
message.attach(MIMEText("wow", "plain"))

# Open the image file and attach it to the email
with open("dashboard/dashboard_plot.png", "rb") as attachment:
    image_part = MIMEImage(attachment.read(), name="dashbaord.jpg")

# Add image to the email
message.attach(image_part)

context = ssl.create_default_context(cafile=certifi.where())

with smtplib.SMTP(smtp_server, port) as server:
    server.starttls(context=context)
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message.as_string())


In [30]:
S = " Search Bar Scrape "
print(S.center(100, '┄'))

┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄ Search Bar Scrape ┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄


'┄┄┄┄50┄┄┄┄'

In [73]:
def progress_bar_once(word, percentage = None, num = 30, ascii = "┄", title = False):
    if title:
        S1 = word.center(num, "=")
        return S1
    else:
        S1 = word.center(num, ascii)
        if percentage: return "├" + S1 + "→  " + str(percentage) + '%'
        else:
            return "├" + S1 + "→  "

In [74]:
progress_bar_once("Completed", 20, num = 30)

'├┄┄┄┄┄┄┄┄┄┄Completed┄┄┄┄┄┄┄┄┄┄┄→  20%'

In [85]:
100/5

20.0

In [99]:

my_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
total_items = len(my_list)

items_completed = 0
for item in my_list:
    
    items_completed += 1
    if items_completed % (total_items // 5) == 0:
        percentage_complete = (items_completed / total_items) * 100
        print(f"Status: {percentage_complete:.2f}% completed")

Status: 20.00% completed
Status: 40.00% completed
Status: 60.00% completed
Status: 80.00% completed
Status: 100.00% completed


In [94]:
for i in range(200):
    if i % 5 == 0:
        print(i)

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195


In [47]:
progress_bar_once("Hello", 54)

'├┄┄┄┄┄┄┄┄┄┄┄┄Hello┄┄┄┄┄┄┄┄┄┄┄┄┄→  54%'

In [43]:
progress_bar_once("Hello")

'├┄┄┄┄┄┄┄┄┄┄┄┄Hello┄┄┄┄┄┄┄┄┄┄┄┄┄→  '

In [50]:
progress_bar_once("Hello", num = 100, title=True)

'===============================================Hello================================================'

In [34]:
from time import sleep


a = ['q','b',1,3]


for i, _ in enumerate(a, start=1):
    max = len(a)
    progress = int(i/max * 50)

    S1 = "Sometest".center(100,"┄")
    print("├" + S1 + "→  " + str(progress) + '%')


    sleep(0.1)


├┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄Sometest┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄→  12%
├┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄Sometest┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄→  25%
├┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄Sometest┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄→  37%
├┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄Sometest┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄→  50%


In [13]:
int(1/3 * 100)

33

In [34]:
a = ['q', 'b', 1, 3, 3, 4, 5]

num_outputs = min(52, len(a))
counter = 0

for i, _ in enumerate(a):
    if counter >= num_outputs:
        break
    
    percentage = ((i + 1) / len(a)) * 100
    print(f"Element {i + 1}: {percentage:.2f}%")
    counter += 1


Element 1: 14.29%
Element 2: 28.57%
Element 3: 42.86%
Element 4: 57.14%
Element 5: 71.43%
Element 6: 85.71%
Element 7: 100.00%


In [23]:
a = ['q', 'b', 1, 3, 3, 4, 5]

for i, _ in enumerate(a):
    percentage = ((i + 1) / len(a)) * 100
    print(f"Element {i + 1}: {percentage:.2f}%")

10.0

In [53]:
100//5

20

In [66]:
def percentage_calc(total_iterations, total_elements, iteration, starts_with = 1):
    percentage_step = 100 / total_iterations

    if starts_with == 0: completion_percentage = round(i+1 * percentage_step, 2)
    else: completion_percentage = round(i * percentage_step, 2)
    return completion_percentage

In [68]:
percentage_calc(total_elements= 100, total_iterations=5, iteration=100)

120.0

In [65]:
a = ['q', 'b', 1, 3, 3, 4, 5]

total_iterations = 6
total_elements = len(a)
percentage_step = 100 / total_iterations

for i in range(1, total_iterations + 1):
    completion_percentage = round(i * percentage_step, 2)
    print(f"Completed: {completion_percentage}%")


Completed: 16.67%
Completed: 33.33%
Completed: 50.0%
Completed: 66.67%
Completed: 83.33%
Completed: 100.0%


In [1]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

def send_email(sender_email, sender_password, receiver_email, subject, body, attachment_path):
    # Set up the SMTP server
    smtp_server = "smtp.gmail.com"
    smtp_port = 587  # For TLS

    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject

    # Add body to email
    message.attach(MIMEText(body, "plain"))

    # Open the file in binary mode
    with open(attachment_path, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)

    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {attachment_path}",
    )

    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()

    # Log in to server and send email
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()  # Secure the connection
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, text)




In [3]:
import datetime as dt
def get_indian_date_time(): return dt.datetime.now(dt.timezone(dt.timedelta(hours=5, minutes=30))).strftime("%dth %b %Y, %I:%M%p")
india_time = get_indian_date_time()

In [4]:
sender_email = "bat463660@gmail.com"
sender_password = "bygc aape tnem adev"
receiver_email = "mohdmafaz200303@gmail.com"
subject = f"Subject: {india_time}.txt"
body = "Text File"

# Path to the attachment file
attachment_path = "/Users/mohamedmafaz/Downloads/Webscraping 3/Output text/2024-03-17 20:13:49.018842.txt"

send_email(sender_email, sender_password, receiver_email, subject, body, attachment_path)

In [8]:
import os
import datetime

def get_file_dates(file):
    """
    Get the last modified and creation dates of the given file.
    """
    try:
        last_modified = time.ctime(os.path.getmtime(file))
        created = time.ctime(os.path.getctime(file))
        return last_modified, created
    except OSError:
        return None, None



In [22]:
import os
import datetime
def modification_date(filename):
    t = os.path.getmtime(filename)
    return datetime.datetime.fromtimestamp(t)

In [23]:
modification_date("Output text")

datetime.datetime(2024, 3, 18, 17, 10, 44, 471628)

In [ ]:
/Users/mohamedmafaz/Downloads/Webscraping 3/Output text/2024-02-04 18:08:11.707601.txt

In [25]:
modification_date("/Users/mohamedmafaz/Downloads/Webscraping 3/Output text/2024-02-04 18:08:11.707601.txt")

datetime.datetime(2024, 3, 8, 18, 13, 9)

In [27]:
file = "/Users/mohamedmafaz/Downloads/Webscraping 3/Output text/2024-02-04 18:08:11.707601.txt"

In [45]:
import os

def delete_files_until_limit(directory, limit):
    size_limit = limit * 1024 * 1024  # 20MB

    def get_file_info(directory):
        file_info = []
        for root, dirs, files in os.walk(directory):
            for file_name in files:
                file_path = os.path.join(root, file_name)
                file_size = os.path.getsize(file_path)
                file_info.append((file_path, file_size))
        file_info.sort(key=lambda x: x[1])
        return file_info

    total_size = sum(file[1] for file in get_file_info(directory))
    files_deleted = 0

    while total_size > size_limit:
        file_info = get_file_info(directory)
        if not file_info:
            print("No more files to delete.")
            break

        file_path, file_size = file_info.pop(0)
        try:
            os.remove(file_path)
            total_size -= file_size
            files_deleted += 1
            print(f"Deleted: {file_path}")
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

    print(f"Total files deleted: {files_deleted}")

# Example usage:
directory_path = "Plots 2"

delete_files_until_limit(directory_path, limit = 0.1)


Deleted: Plots 2/.DS_Store
Deleted: Plots 2/2024-03-17 14:45:49.373238.jpg
Deleted: Plots 2/2024-03-17 15:01:01.467405.jpg
Deleted: Plots 2/2024-03-17 15:09:04.958148.jpg
Deleted: Plots 2/2024-03-17 15:17:56.563081.jpg
Deleted: Plots 2/2024-03-17 15:19:35.660935.jpg
Deleted: Plots 2/2024-03-17 15:03:53.792796.jpg
Deleted: Plots 2/2024-03-17 14:47:13.036850.jpg
Deleted: Plots 2/2024-03-17 15:18:25.562400.jpg
Deleted: Plots 2/2024-03-08 00:17:13.768983.jpg
Deleted: Plots 2/2024-03-17 15:04:15.729400.jpg
Deleted: Plots 2/2024-03-08 01:14:36.603416.jpg
Deleted: Plots 2/2024-03-08 08:46:58.811708.jpg
Deleted: Plots 2/2024-03-17 15:06:51.670299.jpg
Deleted: Plots 2/2024-03-18 17:10:26.959493.jpg
Deleted: Plots 2/2024-03-17 20:13:31.003900.jpg
Deleted: Plots 2/2024-03-08 11:06:22.089434.jpg
Deleted: Plots 2/2024-03-08 08:54:50.441739.jpg
Deleted: Plots 2/2024-03-08 00:09:30.556173.jpg
Total files deleted: 19


In [29]:
created = os.path.getctime(file)
print("Date created: "+time.ctime(created))
print("Date created:",datetime.datetime.fromtimestamp(created))
year,month,day,hour,minute,second=time.localtime(created)[:-3]
print("Date created: %02d/%02d/%d %02d:%02d:%02d"%(day,month,year,hour,minute,second))

Date created: Sun Mar 17 17:02:43 2024
Date created: 2024-03-17 17:02:43.926954
Date created: 17/03/2024 17:02:43
